## Policy Checker AgenticAI Projects

In [ ]:
from dotenv import load_dotenv
import os
from typing import TypedDict
from langchain_openai import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

load_dotenv(override=True)



SQL_SYSTEM_MESSAGE = """
You are a SQL Query Generator. Please note You are SQL Query Generator for Microsoft SQL Server. 

Given an question, create a syntactically correct query to run to help find the answer.
Unless the User specifies in his questions a specific number of examples they want to obtain, always limit your query to at most top 6 results. 
You can order the results by a relevant columns to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description.
Be careful to not query for columns that do not exist.

"""

USER_PROMPT = "Question: {input_text}"


# First define a structured output
class EvaluatorOutput(BaseModel):
    sql_query: str = Field(description="structured SQL query response from the User's question")


llm = ChatOpenAI(model="gpt-4.1", temperature=0)


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


class SQL_chain():

    @staticmethod
    def conversion_in_sql_query(state: State) -> State:
        """llm will generate natural language to structured sql query"""
        
        query_prompt_template = ChatPromptTemplate.from_messages([
            ("system", SQL_SYSTEM_MESSAGE), 
            ("user", USER_PROMPT)
            ])

        structured_llm = llm.with_structured_output(EvaluatorOutput)
        print(structured_llm)

        
        ## format the prompt
        messages = query_prompt_template.format_messages(input_text=state["question"])
        
        ## run llm to generate sql query
        result: EvaluatorOutput = structured_llm.invoke(messages)
        
        ## update state with query
        state["query"] = result.sql_query

        return state





In [23]:
state = {
    "question": "find employees whose salary is more than 40000.",
    "query": "",
    "result": "",
    "answer": ""
}

new_state = SQL_chain.conversion_in_sql_query(state)
print("Generated SQL Query:\n", new_state["query"])


first=RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000266DA7A56D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000266DC7E2510>, root_client=<openai.OpenAI object at 0x00000266DC2A3590>, root_async_client=<openai.AsyncOpenAI object at 0x00000266DA7FF5D0>, model_name='gpt-4.1', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.EvaluatorOutput'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'EvaluatorOutput', 'description': '', 'parameters': {'properties': {'sql_query': {'description': "structured SQL query response from the User's question", 'type': 'string'}}, 'required': ['sql_query'], 'type': 'object'}}}}}, config={}, config_factories=[]) middle=[] last=RunnableBinding(bound=RunnableLambda(...), kwargs={}, config

In [24]:
import pyodbc

def run_sql_query(query: str):
    """ execute the sql query"""

    # SQL server connection details
    server = r"DESKTOP-6SIQQDV\INSTANCE2022"
    database = "ABC_Company"
    username = "sa"
    password = "Sagar@12"


    # Connection string for SQL Authentication
    conn_str = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"Encrypt=no;"
    )

    # connect to database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()


    cursor.execute(query)
    rows = cursor.fetchall()            # ferch rows


    ## convert rows to readable list
    columns = [column[0] for column in cursor.description]
    result = [dict(zip(columns, row)) for row in rows]

    cursor.close()
    conn.close()

    return result

In [25]:
state = {
    "question": "find employees whose salary is more than 40000.",
    "query": "",
    "result": "",
    "answer": ""
}

new_state = SQL_chain.conversion_in_sql_query(state)
print(new_state["query"])


records = run_sql_query(new_state["query"])
new_state["result"] = records

print("\n Query result")
for row in records:
    print(row)

first=RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000266DA7A56D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000266DC7E2510>, root_client=<openai.OpenAI object at 0x00000266DC2A3590>, root_async_client=<openai.AsyncOpenAI object at 0x00000266DA7FF5D0>, model_name='gpt-4.1', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.EvaluatorOutput'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'EvaluatorOutput', 'description': '', 'parameters': {'properties': {'sql_query': {'description': "structured SQL query response from the User's question", 'type': 'string'}}, 'required': ['sql_query'], 'type': 'object'}}}}}, config={}, config_factories=[]) middle=[] last=RunnableBinding(bound=RunnableLambda(...), kwargs={}, config

In [45]:
# from langchain.chains import LLMChain              ## deprecated this line code soon, hence adding "RunnableSequence"
from langchain_core.runnables import RunnableSequence

#from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate


# """
# ### Explain back structured SQL query result in simple language to User
# explain_prompt = PromptTemplate(
#     input_variables = ["question", "result"],
#     template = """
#     You are a helpful data analyst.
#     The user asked: {question}, and here is the SQL query output {result}.
#     Your job to read the question and analyse the SQL query. Explain back result to User in simple, friendly language.
#     Do not show SQL, only explain the meaning, and if any observations about the data.
#     """
# )

# # explain_chain = LLMChain(llm=llm, prompt=explain_prompt)      ## deprecated this line code soon
# explain_chain = explain_prompt | llm
# """



### Explain back structured SQL query result in simple language to User
simple_explain_prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
    You are a helpful assistant who explains SQL query results in simple, natural language.


    Output Style:
    - Briefly restate the user's request
    - Summarize what the results show
    - Bullet list of key rows (Name and Salary or similar fields)
    - One concluding sentence
    - End with a short, helpful conclusion



    Do NOT use tables.
    Just speak like you're summarizing to a colleague.
    """
    ),
    ("user",
    "Question: {question}\n\nQuery Result: {result}")
    ]
)


def explain_result(state):
    explaination = (simple_explain_prompt | llm).invoke({
        "question": state["question"],
        "result": state["result"]
    })
    state["answer"] = explaination.content
    return state

new_state = explain_result(new_state)
print(new_state["answer"])

You asked to find employees whose salary is more than 40,000.

The results show a list of employees who earn above that amount, along with their names and salaries.

Key employees with their salaries:
- Sharad Shinde: 135,000
- Amit Khanna: 120,000
- Alisha Merchant: 115,000
- Sameer Shaikh: 110,000
- Deepak Rane: 98,000
- Rahul Jain: 98,000

All these employees have salaries well above 40,000. Let me know if you need more details or want to filter this list further!
